# Counting Nominations and Confirmations

By [Sean McMinn](http://www.github.com/shmcminn)

Roll Call used the analysis below ot publish an update on the number of nominations President Donald Trump had sent to the Senate, as well as how many the Senate had confirmed. The story can be found [here]().

In [225]:
import pandas as pd
from datetime import datetime

pd.set_option('display.max_colwidth', -1)

additional_june_noms = 18

# Reads in CSV of nominations created by scraping Congress.gov

In [226]:
all_noms = pd.read_csv("all_nominations_confirmations_late_june.csv")
all_noms.head()

,received,confirmed,name,nom_count,agency,duplicate,committee,latest_action,latest_action_date,potus,desc
0,3/21/17,yes,Courtney Elwood,1,Central Intelligence Agency,no,Intelligence,Confirmed by the Senate by Yea-Nay Vote. 67,06/06/2017,Obama,"Courtney Elwood, of Virginia, to be General Counsel of the Central Intelligence Agency, vice Caroline Diane Krass, resigned."
1,1/20/17,yes,Mike Pompeo,1,Central Intelligence Agency,no,Intelligence,Confirmed by the Senate by Yea-Nay Vote. 66,01/23/2017,Obama,"Mike Pompeo, of Kansas, to be Director of the Central Intelligence Agency, vice John Owen Brennan."
2,1/20/17,yes,NaN,1,Civilian,no,Foreign Relations,Confirmed by the Senate by Yea-Nay Vote. 82,05/22/2017,Obama,NaN
3,6/12/17,no,Dawn DeBerry Stump,1,Commodity Futures Trading Commission,no,"Agriculture, Nutrition, and Forestry","Received in the Senate and referred to the Committee on Agriculture, Nutrition, and Forestry. (",06/12/2017,Obama,"Dawn DeBerry Stump, of Texas, to be a Commissioner of the Commodity Futures Trading Commission for a term expiring April 13, 2022, vice Timothy G. Massad resigned."
4,5/16/17,no,Brian D. Quintenz,1,Commodity Futures Trading Commission,no,"Agriculture, Nutrition, and Forestry","Received in the Senate and referred to the Committee on Agriculture, Nutrition, and Forestry. (",05/16/2017,Obama,"Brian D. Quintenz, of Ohio, to be a Commissioner of the Commodity Futures Trading Commission for a term expiring April 13, 2020, vice Scott O'Malia, resigned."


# Gets counts of how many nominees have been sent and confirmed

## Nominees for Foreign Service positions

In [227]:
# number of noms for foreign service

all_noms[all_noms["agency"] == "Foreign Service"]["nom_count"].sum()

649

## Nominees for non-Foreign Service positions

In [228]:
# number noms NOT foreign service

all_noms[all_noms["agency"] != "Foreign Service"]["nom_count"].sum() + additional_june_noms

213

## Confirmations FS positions

In [229]:
# number confirmed foreign service

all_noms[(all_noms["agency"] == "Foreign Service") & (all_noms["confirmed"] == "yes")]["nom_count"].sum()

367

## Confirmations non-FS positions

In [230]:
# number confirmed NOT foreign service

all_noms[(all_noms["agency"] != "Foreign Service") & (all_noms["confirmed"] == "yes")]["nom_count"].sum()

49

## Counts number of nominees received each month

In [231]:
# breadown noms by month

all_noms['received'] = pd.to_datetime(all_noms["received"])
all_noms.index = all_noms["received"]

monthly_noms = all_noms.resample('M').sum()



print(str(additional_june_noms) + " additional for June")
print(monthly_noms)
print("total noms (incl addtl june noms): " + str(monthly_noms.sum() + additional_june_noms))

18 additional for June
            nom_count
received             
2017-01-31  27       
2017-02-28  7        
2017-03-31  15       
2017-04-30  394      
2017-05-31  46       
2017-06-30  355      
total noms (incl addtl june noms): nom_count    862
dtype: int64


## Counts number of nominees in June vs first months

In [232]:
# breakdown  noms in June

print(str(int(monthly_noms[monthly_noms.index == "2017-06-30"]["nom_count"]) + additional_june_noms) + " total noms in jun")

print(str(monthly_noms[monthly_noms.index != "2017-06-30"]["nom_count"].sum()) + " in first mos")

monthly_noms_fs = all_noms[all_noms["agency"] == "Foreign Service"].resample('M').sum()

print(str(int(monthly_noms_fs[monthly_noms_fs.index == "2017-06-30"]["nom_count"])) + " FS noms in jun")

373 total noms in jun
489 in first mos
278 FS noms in jun


## Counts number of confirmations by month for all noms and just FS noms

In [233]:
# breadown confirmations by month

all_noms['latest_action_date'] = pd.to_datetime(all_noms["latest_action_date"])
all_noms.index = all_noms["latest_action_date"]
print(all_noms[all_noms["confirmed"] == "yes"].resample('M').sum())

print("\n\nJust FS:")
print(all_noms[(all_noms["confirmed"] == "yes") & (all_noms["agency"] == "Foreign Service")].resample('M').sum())

print("\n\naverage non FS confs per month is " + str(all_noms[(all_noms["confirmed"] == "yes") & (all_noms["agency"] != "Foreign Service")].resample('M').sum()["nom_count"].mean()))

                    nom_count
latest_action_date           
2017-01-31          5        
2017-02-28          10       
2017-03-31          8        
2017-04-30          5        
2017-05-31          361      
2017-06-30          27       


Just FS:
                    nom_count
latest_action_date           
2017-05-31          347      
2017-06-30          20       


average non FS confs per month is 8.16666666667


## Calculates number of people nominated but not confirmed

In [235]:
#backlog

withdraws = 4

all_noms["nom_count"].sum() + additional_june_noms - all_noms[all_noms["confirmed"] == "yes"]["nom_count"].sum() - withdraws

442